In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from scipy.stats import spearmanr
from scipy import stats


In [35]:
data = pd.read_excel("E:\Goal 3\Analysis\KAP Survey Tool\KAP Survey Tool.xlsx")

#### Data Cleaning

In [36]:
hospital_levels = {
    "CHUK": "Referral/Teaching Hospital",
    "Masaka DH": "District Hospital",
    "Nyamata DH": "Level 2 Teaching Hospital",
    "Kirehe DH": "District Hospital",
    "Butaro DH": "Level 2 Teaching Hospital",
    "Nemba DH": "District Hospital",
    "CHUB": "Referral/Teaching Hospital",
    "Kabutare DH": "District Hospital",
    "Kibogora DH": "Level 2 Teaching Hospital",
    "Gisenyi DH": "District Hospital"
}
data['hospital_level'] = data['Hospital name'].map(hospital_levels)
# Replace values in the entire DataFrame
data['cadre'].replace({1: 'Nurse', 
            2: 'Nurse-midwife', 
            3: 'Nurse in charge', 4: 'Resident doctor', 5: 'MD'}, inplace=True)

C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\3959201680.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['cadre'].replace({1: 'Nurse',


### Used Functions

In [37]:
def print_and_export_frequency_table(df, column_name, output_filename="frequency_table.xlsx"):
    """
    Prints a frequency table for a given column in a dataframe using PrettyTable and exports it to an Excel file.
    """
    # Get value counts and percentages
    value_counts = df[column_name].value_counts()
    percentages = (value_counts / len(df)) * 100
    
    # Create PrettyTable
    table = PrettyTable()
    table.field_names = [column_name, "Count", "Percentage"]
    
    # Prepare data for export
    export_data = []
    for category, count in value_counts.items():
        percentage = f"{percentages[category]:.2f}%"
        table.add_row([category, count, percentage])
        export_data.append([category, count, percentages[category]])
    
    print(table)
    
    # Convert to DataFrame and export to Excel
    export_df = pd.DataFrame(export_data, columns=[column_name, "Count", "Percentage"])
    export_df.to_excel(output_filename, index=False)
    print(f"Frequency table exported to {output_filename}")
    
def compare_groups(df, group_col, score_col):
    """
    Perform One-way ANOVA and Kruskal-Wallis test on a given score column across different groups.

    Parameters:
    - df: pandas DataFrame
    - group_col: str, column name representing the categorical grouping variable
    - score_col: str, column name representing the continuous score variable

    Returns:
    - None (prints test results)
    """
    print(f"\n=== Analysis of {score_col} by {group_col} ===")
    
    # Drop missing values
    df = df.dropna(subset=[group_col, score_col])
    
    # Extract unique groups
    groups = df[group_col].unique()
    
    # Step 1: Check normality for each group using Shapiro-Wilk test
    normality_results = {}
    grouped_data = []
    
    for group in groups:
        group_data = df[df[group_col] == group][score_col]
        grouped_data.append(group_data)
        
        if len(group_data) > 3:  # Shapiro-Wilk requires at least 3 values
            stat, p_value = stats.shapiro(group_data)
            normality_results[group] = p_value
        else:
            normality_results[group] = None  # Not enough data
    
    print("\nNormality Test (Shapiro-Wilk) Results per Group:")
    for group, p in normality_results.items():
        if p is not None:
            print(f"Group {group}: p-value = {p:.4f} {'(Normal)' if p > 0.05 else '(Not Normal)'}")
        else:
            print(f"Group {group}: Not enough data for test")

    # Step 2: Check homogeneity of variances using Levene’s test
    levene_stat, levene_p = stats.levene(*grouped_data, center='median')

    print(f"\nLevene’s Test for Homogeneity of Variances: p-value = {levene_p:.4f}")
    if levene_p > 0.05:
        print("Variances are equal (assumption met).")
    else:
        print("Variances are not equal (consider a non-parametric alternative like Kruskal-Wallis).")

    # Step 3: Perform One-way ANOVA
    anova_stat, anova_p = stats.f_oneway(*grouped_data)

    print(f"\nOne-way ANOVA Results: p-value = {anova_p:.4f}")
    if anova_p < 0.05:
        print("Conclusion: Reject H0. At least one group has a different mean score.")
    else:
        print("Conclusion: Fail to reject H0. No significant difference among groups.")

    # Step 4: Perform Kruskal-Wallis test (if assumptions are violated)
    kruskal_stat, kruskal_p = stats.kruskal(*grouped_data)

    print(f"\nKruskal-Wallis Test Results: p-value = {kruskal_p:.4f}")
    if kruskal_p < 0.05:
        print("Conclusion: Reject H0. At least one group has a different median score.")
    else:
        print("Conclusion: Fail to reject H0. No significant difference in median scores among groups.")
        
def spearman_correlation(df, experience_col, score_col):
    """
    Perform Spearman Correlation test between years of experience and a given score column.

    Parameters:
    - df: pandas DataFrame
    - experience_col: str, column name representing years of experience
    - score_col: str, column name representing the continuous score variable

    Returns:
    - None (prints correlation results)
    """
    # Drop missing values
    df = df.dropna(subset=[experience_col, score_col])

    # Perform Spearman correlation
    spearman_corr, spearman_p = spearmanr(df[experience_col], df[score_col])

    # Output results
    print(f"\nSpearman Correlation Test: {experience_col} vs {score_col}")
    print(f"Spearman Correlation Coefficient: {spearman_corr:.4f}")
    print(f"p-value: {spearman_p:.4f}")

    # Interpretation
    if spearman_p < 0.05:
        print("Conclusion: Significant correlation.")
    else:
        print("Conclusion: No significant correlation.")



### 1. Knowledge

In [38]:
data.columns

Index(['Unnamed: 0', 'record_id', 'form_1_timestamp', 'cadre',
       'years_of_service_in_this_r', 'hospital_level', 'Hospital name',
       'airway_management', 'breathing_and_ventilation',
       'circulation_and_shock', 'circulation_and_shock_2',
       'cardiopulmonary_resuscitation', 'sepsis_recognition',
       'sepsis_recognition_2', 'hypoglycemia_management', 'seizure_management',
       'fluid_resuscitation', 'thermoregulation', 'medication_dosing',
       'confidence_in_skills', 'team_collaboration', 'training_adequacy',
       'importance_of_protocols', 'resource_availability',
       'continuing_education', 'stress_management',
       'importance_of_early_recognition', 'a_multidisciplinary_approa',
       'family_involvement', 'airway_assessment', 'use_of_bag_valve_mask',
       'shock_management', 'cpr_performance', 'sepsis_protocols',
       'hypoglycemia_treatment', 'seizure_response',
       'fluid_resuscitation_practices', 'how_often_do_you_follow_be',
       'medicat

In [39]:
data_know = data[['hospital_level','Hospital name','airway_management_score',
       'ventilation_score', 'circulation_an_shock_score', 'circulation_an_shock_score_2', 'cardiopulmonary_score',
       'sepsis_recognitio_score', 'sepsis_recognitio_score_2', 'hypoglycemia_score', 'seizur_management_score',
       'fluid_resuscitatin_score', 'thermoregulatin_score','medication_dosin_score']]

In [40]:
# Replace values in the entire DataFrame
data_know.replace({0: 'Incorrect response', 
            1: 'Partially correct response', 
            2: 'Completely correct'}, inplace=True)

C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\4166274125.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_know.replace({0: 'Incorrect response',


In [41]:
# data_know

In [42]:
print_and_export_frequency_table(data_know, "medication_dosin_score", "frequency_table.xlsx")

+----------------------------+-------+------------+
|   medication_dosin_score   | Count | Percentage |
+----------------------------+-------+------------+
| Partially correct response |   68  |   53.97%   |
|     Incorrect response     |   47  |   37.30%   |
|     Completely correct     |   11  |   8.73%    |
+----------------------------+-------+------------+
Frequency table exported to frequency_table.xlsx


C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\2798460452.py:24: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  export_df.to_excel(output_filename, index=False)


### 3. Attitudes

In [43]:
# data.columns

In [44]:
data_att = data[['hospital_level','Hospital name','confidence_in_skills', 'team_collaboration', 'training_adequacy',
       'importance_of_protocols', 'resource_availability',
       'continuing_education', 'stress_management',
       'importance_of_early_recognition', 'a_multidisciplinary_approa',
       'family_involvement',]]

In [45]:
# Replace values in the entire DataFrame
data_att.replace({1: 'Strongly Disagree', 
            2: 'Disagree', 
            3: 'Neutral', 4: 'Agree', 5: 'Strongly Agree'}, inplace=True)

C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\3806833358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_att.replace({1: 'Strongly Disagree',


In [46]:
# print_and_export_frequency_table(data_att, "family_involvement", "frequency_table.xlsx")

### 3. Practices

In [47]:
data_prac = data[['hospital_level','Hospital name','airway_assessment', 'use_of_bag_valve_mask',
       'shock_management', 'cpr_performance', 'sepsis_protocols', 'hypoglycemia_treatment', 'seizure_response',
       'fluid_resuscitation_practices', 'how_often_do_you_follow_be', 'medication_dosage_calculation']]

In [48]:
data_prac.replace({1: 'Always', 
            2: 'Often', 
            3: 'Sometimes', 4: 'Rarely', 5: 'Never'}, inplace=True)


C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\237127318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_prac.replace({1: 'Always',


In [49]:
data_prac
print_and_export_frequency_table(data_prac, "use_of_bag_valve_mask", "frequency_table.xlsx")

+-----------------------+-------+------------+
| use_of_bag_valve_mask | Count | Percentage |
+-----------------------+-------+------------+
|       Sometimes       |   54  |   42.86%   |
|         Often         |   31  |   24.60%   |
|         Always        |   27  |   21.43%   |
|         Rarely        |   12  |   9.52%    |
|         Never         |   2   |   1.59%    |
+-----------------------+-------+------------+
Frequency table exported to frequency_table.xlsx


C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\2798460452.py:24: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  export_df.to_excel(output_filename, index=False)


#### Association of KAP Outcome Vs Cadre

In [50]:
def add_summation_column(df, column_list, new_column_name):
    """
    Adds a new column to the DataFrame that contains the sum of specified columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_list (list): List of column names to sum.
    new_column_name (str): The name of the new column that will store the summation.

    Returns:
    pd.DataFrame: Updated DataFrame with the new summation column.
    """
    if not all(col in df.columns for col in column_list):
        raise ValueError("One or more columns in column_list do not exist in the DataFrame.")

    df[new_column_name] = df[column_list].sum(axis=1)
    return df

In [51]:
data = add_summation_column(data, ['airway_management_score', 'ventilation_score', 'circulation_an_shock_score', 'circulation_an_shock_score_2', 'cardiopulmonary_score','sepsis_recognitio_score', 'sepsis_recognitio_score_2', 'hypoglycemia_score', 'seizur_management_score',
       'fluid_resuscitatin_score', 'thermoregulatin_score','medication_dosin_score'], "Total Knowledge score")

In [52]:
data = add_summation_column(data, ['confidence_in_skills', 'team_collaboration', 'training_adequacy','importance_of_protocols', 'resource_availability','continuing_education', 'stress_management',
       'importance_of_early_recognition', 'a_multidisciplinary_approa','family_involvement',], "Total Attitude score")

In [53]:
data_prac.replace({'Always': 5, 
            'Often': 4, 
            'Sometimes': 3 , 'Rarely': 2, 'Never': 1}, inplace=True)
data_prac = add_summation_column(data_prac, ['airway_assessment', 'use_of_bag_valve_mask', 'shock_management', 'cpr_performance', 'sepsis_protocols', 'hypoglycemia_treatment', 'seizure_response',
       'fluid_resuscitation_practices', 'how_often_do_you_follow_be', 'medication_dosage_calculation'], "Total Practice score")
data["Total Practice score"] = data_prac["Total Practice score"]

C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\1787448539.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_prac.replace({'Always': 5,
C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\1787448539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_prac.replace({'Always': 5,
C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\1485862288.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [54]:
data

,Unnamed: 0,record_id,form_1_timestamp,cadre,years_of_service_in_this_r,hospital_level,Hospital name,airway_management,breathing_and_ventilation,circulation_and_shock,...,sepsis_recognitio_score,sepsis_recognitio_score_2,hypoglycemia_score,seizur_management_score,fluid_resuscitatin_score,thermoregulatin_score,medication_dosin_score,Total Knowledge score,Total Attitude score,Total Practice score
0,0,1,2024-11-25 13:37:59,MD,0.0,District Hospital,Masaka DH,Make sure the airway is patent,Head tilt /chin lift,"Cool and clammy skin, bluish is discoloration ...",...,2,2,0,0,1,2,0,13.5,49,40
1,1,2,2024-11-25 14:51:58,Nurse,6.0,District Hospital,Masaka DH,We keep air way clean by putting baby in neutr...,"Neutral position, Taking a valve mask like mo...","General cyanosis, severe respiratory distress",...,1,1,2,0,0,2,0,10.0,50,50
2,2,3,2024-11-25 15:20:17,Nurse,20.0,Level 2 Teaching Hospital,Nyamata DH,degager les voies respiratoires mettre l'enf...,asepsie,convulsions; Difficulté respiratoire léthargi...,...,1,0,1,1,0,0,0,6.0,34,46
3,3,4,2024-11-25 15:28:56,Nurse,15.0,Level 2 Teaching Hospital,Nyamata DH,you can assess if the baby has obstruction in ...,keep the baby in good position (en hyperextens...,"dehydration, extremitie froid, tachypnea, hypo...",...,1,0,1,0,0,0,0,8.0,43,47
4,4,5,2024-11-25 15:11:35,Nurse,16.0,Level 2 Teaching Hospital,Nyamata DH,taking vital signs,"mouth open, baby in neutral position , mask co...","cold skin, respiratory destress, bradycardia",...,1,1,1,1,0,1,0,8.0,47,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,340,341,2024-12-13 13:40:30,Nurse,16.0,Referral/Teaching Hospital,CHUK,If airways are not obstructed,Cover nose and mouth,"palor,dehydration Action:You give transfusio...",...,1,2,2,0,0,2,1,13.0,45,28
122,345,346,2024-12-16 14:02:50,Nurse,14.0,Referral/Teaching Hospital,CHUK,Positioning patient Clearing airway,Performing bag valve mask Positioning a patie...,"Poor perfusion (cold extremities,mottled,skin ...",...,2,2,2,2,2,2,2,22.0,46,48
123,351,352,2024-12-17 13:30:29,Nurse,10.0,Referral/Teaching Hospital,CHUK,"position of the neck , suctioning if there is ...",is to use C technic,"tachycardia, poor perfusion ( no pulses on pal...",...,2,2,2,2,1,1,1,17.0,40,46
124,353,354,2024-12-17 13:31:12,Resident doctor,3.0,Referral/Teaching Hospital,CHUK,"assessment of airways patency, work of breathing","making C shaped mask application on the face, ...",altered mental stutus and tachycardia with red...,...,0,2,1,1,1,2,1,16.0,45,42


#### Knowledge

In [55]:
data_ass = data[['Hospital name','hospital_level','cadre','years_of_service_in_this_r', 'Total Knowledge score', 'Total Attitude score', 'Total Practice score']]
compare_groups(data_ass, 'cadre','Total Knowledge score')


=== Analysis of Total Knowledge score by cadre ===

Normality Test (Shapiro-Wilk) Results per Group:
Group MD: p-value = 0.4897 (Normal)
Group Nurse: p-value = 0.0699 (Normal)
Group Nurse-midwife: p-value = 0.3960 (Normal)
Group Nurse in charge: p-value = 0.0747 (Normal)
Group Resident doctor: Not enough data for test

Levene’s Test for Homogeneity of Variances: p-value = 0.0047
Variances are not equal (consider a non-parametric alternative like Kruskal-Wallis).

One-way ANOVA Results: p-value = 0.1751
Conclusion: Fail to reject H0. No significant difference among groups.

Kruskal-Wallis Test Results: p-value = 0.1609
Conclusion: Fail to reject H0. No significant difference in median scores among groups.


#### Attitude

In [56]:
compare_groups(data_ass, 'cadre','Total Attitude score')


=== Analysis of Total Attitude score by cadre ===

Normality Test (Shapiro-Wilk) Results per Group:
Group MD: p-value = 0.8400 (Normal)
Group Nurse: p-value = 0.0000 (Not Normal)
Group Nurse-midwife: p-value = 0.3100 (Normal)
Group Nurse in charge: p-value = 0.0002 (Not Normal)
Group Resident doctor: Not enough data for test

Levene’s Test for Homogeneity of Variances: p-value = 0.1576
Variances are equal (assumption met).

One-way ANOVA Results: p-value = 0.5184
Conclusion: Fail to reject H0. No significant difference among groups.

Kruskal-Wallis Test Results: p-value = 0.2392
Conclusion: Fail to reject H0. No significant difference in median scores among groups.


#### Practice

In [57]:
compare_groups(data_ass, 'cadre','Total Practice score')


=== Analysis of Total Practice score by cadre ===

Normality Test (Shapiro-Wilk) Results per Group:
Group MD: p-value = 0.3607 (Normal)
Group Nurse: p-value = 0.0000 (Not Normal)
Group Nurse-midwife: p-value = 0.0717 (Normal)
Group Nurse in charge: p-value = 0.1055 (Normal)
Group Resident doctor: Not enough data for test

Levene’s Test for Homogeneity of Variances: p-value = 0.3946
Variances are equal (assumption met).

One-way ANOVA Results: p-value = 0.2803
Conclusion: Fail to reject H0. No significant difference among groups.

Kruskal-Wallis Test Results: p-value = 0.2842
Conclusion: Fail to reject H0. No significant difference in median scores among groups.


### Association of KAP Outcome Vs Hospital level

In [58]:
compare_groups(data_ass, 'hospital_level','Total Knowledge score')


=== Analysis of Total Knowledge score by hospital_level ===

Normality Test (Shapiro-Wilk) Results per Group:
Group District Hospital: p-value = 0.1711 (Normal)
Group Level 2 Teaching Hospital: p-value = 0.8375 (Normal)
Group Referral/Teaching Hospital: p-value = 0.0825 (Normal)

Levene’s Test for Homogeneity of Variances: p-value = 0.0214
Variances are not equal (consider a non-parametric alternative like Kruskal-Wallis).

One-way ANOVA Results: p-value = 0.0000
Conclusion: Reject H0. At least one group has a different mean score.

Kruskal-Wallis Test Results: p-value = 0.0000
Conclusion: Reject H0. At least one group has a different median score.


In [59]:
compare_groups(data_ass, 'hospital_level','Total Attitude score')


=== Analysis of Total Attitude score by hospital_level ===

Normality Test (Shapiro-Wilk) Results per Group:
Group District Hospital: p-value = 0.0000 (Not Normal)
Group Level 2 Teaching Hospital: p-value = 0.0000 (Not Normal)
Group Referral/Teaching Hospital: p-value = 0.0000 (Not Normal)

Levene’s Test for Homogeneity of Variances: p-value = 0.1365
Variances are equal (assumption met).

One-way ANOVA Results: p-value = 0.0931
Conclusion: Fail to reject H0. No significant difference among groups.

Kruskal-Wallis Test Results: p-value = 0.0399
Conclusion: Reject H0. At least one group has a different median score.


In [60]:
compare_groups(data_ass, 'hospital_level','Total Practice score')


=== Analysis of Total Practice score by hospital_level ===

Normality Test (Shapiro-Wilk) Results per Group:
Group District Hospital: p-value = 0.0000 (Not Normal)
Group Level 2 Teaching Hospital: p-value = 0.0113 (Not Normal)
Group Referral/Teaching Hospital: p-value = 0.0000 (Not Normal)

Levene’s Test for Homogeneity of Variances: p-value = 0.8867
Variances are equal (assumption met).

One-way ANOVA Results: p-value = 0.7201
Conclusion: Fail to reject H0. No significant difference among groups.

Kruskal-Wallis Test Results: p-value = 0.6253
Conclusion: Fail to reject H0. No significant difference in median scores among groups.


### Association of KAP Outcome Vs Years of experience

#### 1. Knowledge

In [61]:
spearman_correlation(data_ass, "years_of_service_in_this_r", "Total Knowledge score")


Spearman Correlation Test: years_of_service_in_this_r vs Total Knowledge score
Spearman Correlation Coefficient: -0.0022
p-value: 0.9807
Conclusion: No significant correlation.


#### 2. Attitude

In [62]:
spearman_correlation(data_ass, "years_of_service_in_this_r", "Total Attitude score")


Spearman Correlation Test: years_of_service_in_this_r vs Total Attitude score
Spearman Correlation Coefficient: -0.0875
p-value: 0.3297
Conclusion: No significant correlation.


#### 3. Practice

In [63]:
spearman_correlation(data_ass, "years_of_service_in_this_r", "Total Practice score")


Spearman Correlation Test: years_of_service_in_this_r vs Total Practice score
Spearman Correlation Coefficient: 0.0280
p-value: 0.7555
Conclusion: No significant correlation.


In [64]:
data_ass.columns

Index(['Hospital name', 'hospital_level', 'cadre',
       'years_of_service_in_this_r', 'Total Knowledge score',
       'Total Attitude score', 'Total Practice score'],
      dtype='object')

In [65]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from prettytable import PrettyTable

def summarize_scores(df, group_col, score_cols, output_file="summary.xlsx"):
    """
    Summarizes scores by computing mean, median, standard deviation, IQR, and confidence intervals (25%, 50%, 95%)
    for each group in the specified column. Outputs results as an Excel file.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        group_col (str): The column name used to group data.
        score_cols (list): List of column names containing scores.
        output_file (str): The name of the Excel file to save the results.

    Returns:
        PrettyTable: A table with statistical summaries (also saves an Excel file).
    """
    # Grouping the data by the specified column
    grouped = df.groupby(group_col)[score_cols]

    # Creating a PrettyTable instance
    table = PrettyTable()
    table.field_names = [
        "Group", "Score Column", "Mean", "Median", "Std Dev", "IQR", 
        "25% CI", "50% CI", "95% CI"
    ]

    # Data storage for Excel export
    summary_data = []

    # Looping through each group and score column to compute statistics
    for group, data in grouped:
        for col in score_cols:
            mean_val = data[col].mean()
            median_val = data[col].median()
            std_val = data[col].std()
            iqr_val = data[col].quantile(0.75) - data[col].quantile(0.25)  # IQR = Q3 - Q1
            
            # Compute Confidence Intervals
            n = len(data[col].dropna())  # Exclude NaNs
            se = std_val / np.sqrt(n) if n > 0 else 0  # Standard Error
            
            ci_25 = (mean_val - norm.ppf(0.875) * se, mean_val + norm.ppf(0.875) * se)
            ci_50 = (mean_val - norm.ppf(0.75) * se, mean_val + norm.ppf(0.75) * se)
            ci_95 = (mean_val - norm.ppf(0.975) * se, mean_val + norm.ppf(0.975) * se)

            # Add row to the PrettyTable
            table.add_row([
                group, col, round(mean_val, 2), round(median_val, 2), round(std_val, 2), round(iqr_val, 2),
                f"({round(ci_25[0], 2)}, {round(ci_25[1], 2)})",
                f"({round(ci_50[0], 2)}, {round(ci_50[1], 2)})",
                f"({round(ci_95[0], 2)}, {round(ci_95[1], 2)})"
            ])

            # Append row to summary_data for Excel export
            summary_data.append([
                group, col, mean_val, median_val, std_val, iqr_val, 
                ci_25[0], ci_25[1], ci_50[0], ci_50[1], ci_95[0], ci_95[1]
            ])

    # Convert summary_data to a DataFrame for Excel export
    summary_df = pd.DataFrame(summary_data, columns=[
        "Group", "Score Column", "Mean", "Median", "Std Dev", "IQR", 
        "25% CI Lower", "25% CI Upper", "50% CI Lower", "50% CI Upper", "95% CI Lower", "95% CI Upper"
    ])

    # Export to Excel
    summary_df.to_excel(output_file, index=False)

    print(f"Summary statistics exported to {output_file}")
    
    return table

# Example usage:
# df = pd.read_csv("your_data.csv")  # Load your data
print(summarize_scores(data, "hospital_level", ["Total Knowledge score", "Total Attitude score", "Total Practice score"], "summary.xlsx"))


Summary statistics exported to summary.xlsx
+----------------------------+-----------------------+-------+--------+---------+------+----------------+----------------+----------------+
|           Group            |      Score Column     |  Mean | Median | Std Dev | IQR  |     25% CI     |     50% CI     |     95% CI     |
+----------------------------+-----------------------+-------+--------+---------+------+----------------+----------------+----------------+
|     District Hospital      | Total Knowledge score | 12.59 |  12.5  |   4.44  | 5.0  | (11.94, 13.24) | (12.21, 12.97) | (11.48, 13.69) |
|     District Hospital      |  Total Attitude score | 42.92 |  45.0  |   6.6   | 7.0  | (41.96, 43.88) | (42.35, 43.48) | (41.28, 44.56) |
|     District Hospital      |  Total Practice score | 40.58 |  42.0  |   8.09  | 7.75 | (39.4, 41.76)  | (39.89, 41.27) | (38.57, 42.59) |
| Level 2 Teaching Hospital  | Total Knowledge score | 11.38 |  12.0  |   3.19  | 5.0  | (10.77, 11.98) | (11.02, 11

C:\Users\ishim\AppData\Local\Temp\ipykernel_27824\870449628.py:70: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  summary_df.to_excel(output_file, index=False)


In [66]:
# mean_scores